## **IMPORTS**

In [1]:
import pandas as pd
from string import ascii_uppercase as alphabet
import pickle
from bs4 import BeautifulSoup
import requests

- trebam procitati potrebne podatke sa stranice u kojoj su osnovni podaci za svjetsko prvenstvo 2022 bez ikakvih rezultata, to radim sa metodom read_html koja je dostupna sa pandas

### **WEBSCRAPING WORLD CUP 2022 GROUPS**

In [2]:
all_tables = pd.read_html("https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup")

In [3]:
#all_tables[12]
#all_tables[19]
all_tables[26]
all_tables[61]

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Portugal,0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Ghana,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Uruguay,0,0,0,0,0,0,0,0,NaN
3,4,South Korea,0,0,0,0,0,0,0,0,NaN


-  dobivanje svih grupa i primjena promjena na dataframe

In [4]:
dict_table = {}
for letter, i in zip(alphabet, range(12, 68, 7)):
    df = all_tables[i] # vucem svaku od grupa u dataframe
    df.rename(columns={"Teamvte":"Team"}, inplace = True) # renameam stupac da je jasniji
    df.pop("Qualification")
    dict_table[f"Group {letter}"] = df

In [5]:
dict_table.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

In [6]:
dict_table["Group A"]

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Qatar (H),0,0,0,0,0,0,0,0
1,2,Ecuador,0,0,0,0,0,0,0,0
2,3,Senegal,0,0,0,0,0,0,0,0
3,4,Netherlands,0,0,0,0,0,0,0,0


- spremanje dobivenih podataka 

In [7]:
with open("world_cup_groups_2022","wb") as output:
    pickle.dump(dict_table, output)

### **WEBSCRAPING ALL THE HISTORICAL DATA**

In [8]:
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
        1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
        2018]

In [10]:
def get_matches(year):
    if year == 2022:
        website = f"https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup"
    else:
        website = f"https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(website, headers=headers)
    content = response.text
    soup = BeautifulSoup(content, "lxml")
    matches = soup.find_all("div", class_="footballbox")
    home = []
    score = []
    away = []
    for match in matches:
        home.append(match.find("th", class_="fhome").get_text())
        score.append(match.find("th", class_="fscore").get_text())
        away.append(match.find("th", class_="faway").get_text())
    dict_football = {"home": home, "score": score, "away": away}
    df_football = pd.DataFrame(dict_football)
    df_football["year"] = year
    return df_football

In [11]:
get_matches("2018")

,home,score,away,year
0,Russia,5–0,Saudi Arabia,2018
1,Egypt,0–1,Uruguay,2018
2,Russia,3–1,Egypt,2018
3,Uruguay,1–0,Saudi Arabia,2018
4,Uruguay,3–0,Russia,2018
...,...,...,...,...
59,Russia,2–2 (a.e.t.),Croatia,2018
60,France,1–0,Belgium,2018
61,Croatia,2–1 (a.e.t.),England,2018
62,Belgium,2–0,England,2018


In [12]:
fifa = [get_matches(year) for year in years]

In [13]:
df_fifa = pd.concat(fifa, ignore_index = True)

In [14]:
df_fifa.to_csv("fifa_worldcup_historical_data.csv", index=False)

### **GETTING 2022 FIXTURES**

In [15]:
df_fixture = get_matches(2022)

In [16]:
df_fixture.to_csv("fifa_worldcup_2022_fixtures.csv", index=False)